## カウントベース手法の問題点

カウントベース手法では、共起行列->ppmi->SVDという処理を経てテキストデータ全体の統計データを一度に処理していた。そのため大きなデータでは途方もない時間がかかる。特にSVDのオーダーは$O(n^3)$となるので、特に時間がかかる

そこで、ニューラルネットワークのミニバッチ法を使って少しずつ学習していく方法（推論ベース手法）を見ていく

## 推論ベース手法
推論ベースの手法では、コンテキスト(周囲の単語)が与えられたとき、自分自身をコンテキストから推論する。

「you ? goodbye and i say hello」?を周囲から推測する

推論ベースもカウントベースと同じく、分布仮説に基づいている

推論ベースで単語を扱う場合、固定長ベクトルに変換する必要がある。そこで、単語一つ一つをone-hot表現へ変換する

say->$(1,0,0,0,0,0,0)$

you->$(0,0,1,0,0,0,0)$

ベクトル長を固定することで、ニューロンの数も固定することができる。

ここでは、単語数７の文をコーパスとして実装してみる

In [1]:
import numpy as np
import sys
sys.path.append("../../deep-learning-from-scratch-2")
from common.layers import MatMul

In [4]:
c = np.array([[1,0,0,0,0,0,0]]) # 単語IDが０の単語
W = np.random.randn(7, 3)
# h = np.dot(c, W)
layer = MatMul(W)
h = layer.forward(c)
h

array([[-0.2117446 , -1.32101092,  0.98811033]])

## CBOWモデル
ではここから、ニューラルネットワークを構築していく。

モデルとしては、word2vecで提案されている　continuous bag-of-words(CBOW)というモデルを使う。

CBOWとは、コンテキスト(周辺単語のリスト）を入力、ただしone-hot表現の。

CBOWの入力層は２つあり、中間層を経由して出力層のニューロンへ向かう。周辺単語を今は２としているため、入力層は２つだが、周辺４単語であれば４つの入力層がある。

学習を進めると、入力層から中間層への重みが更新されていくが、この重み行列が単語の分散表現そのものになる。行列の各行は対応する単語の意味をうまくエンコードしたものとみなすことも出来る。

入力層７に対して、中間層は３にする。そうすることで、コンパクトにデータを圧縮することになり、つまり密なベクトルへの変換していることになる。

In [7]:
# 推論処理
import sys
sys.path.append("../../deep-learning-from-scratch-2")
import numpy as np
from common.layers import MatMul

c0 = np.array([[1,0,0,0,0,0,0]])
c1 = np.array([[0,0,1,0,0,0,0]])

W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
s

array([[ 0.74944059,  0.08903261, -0.53344934, -0.03440409,  0.52755354,
        -0.08417425, -0.5304984 ]])

In [8]:
# 順伝搬を実装。前処理
import sys
sys.path.append("../../deep-learning-from-scratch-2")
import numpy as np
from common.util import preprocess

text = "you say goodbye and i say hello."
corpus, word_to_id, id_to_word = preprocess(text)

In [16]:
# コーパスからコンテキストとターゲットのリストを作るメソッドを実装
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]# 1番目から最後の一つ手前まで
    contexts = []
    
    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)
    return np.array(contexts), np.array(target)

In [17]:
contexts, target = create_contexts_target(corpus, window_size=1)

In [18]:
contexts

array([[0, 2],
       [1, 3],
       [2, 4],
       [3, 1],
       [4, 5],
       [1, 6]])

In [19]:
target

array([1, 2, 3, 4, 1, 5])

In [20]:
# one-hotベクトルへ変換
from common.util import convert_one_hot

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)